# 3.0 ~ 3.2 

- LLMs and Chat Models
- Predict Messsages
- Prompt Templates

In [23]:
# from dotenv import load_dotenv
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# StreamingStdOutCallbackHandler은 문자가 생성될 때 마다 출력을 해주는 콜백 핸들러.
chat = ChatOpenAI(temperature=0.5, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

In [ ]:
# t = PromptTemplate(
#   template = "What is the capital of {country}?",
#   input_variables= ["country"]
# )

t = PromptTemplate.from_template("What is the capital of {country}?")

t.format(country="Korea")

In [ ]:
template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}?")

# load_dotenv()

prompt = template.format(country_a="France", country_b="Germany")

# llm = OpenAI(model_name="gpt-3.5-turbo-1106")

chat.predict(prompt)

In [ ]:
# a = llm.predict("How many planets are in the solar system?")
b = chat.predict("How many planets are in the solar system?")

In [ ]:
#a
b

'There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [7]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
  SystemMessage(
      content="You are a geography expert. And you only reply in Italian.",
    ),
  AIMessage(content="Ciao, mi chiamo Paolo"),
  HumanMessage(content="What is the distance between Mexico and the Thailand?"),
]

chat.predict_messages(messages)


AIMessage(content='La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri.')

In [13]:
# Prompt Template

template = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}?"),
  ]
)

prompt = template.format_messages(language="Greek", name="Socrates", country_a="Mexico", country_b="Thailand")

chat.predict_messages(prompt)

AIMessage(content='Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

## Note

프롬프트 템플릿 작성 후 format에서 placeholder에 값을 넣지 않으면 에러 발생

```python
template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}?")

# load_dotenv()

# llm = OpenAI(model_name="gpt-3.5-turbo-1106")
chat = ChatOpenAI(temperature=0.5)

template.format()
```

![스크린샷 2024-03-23 오후 7 54 14](https://github.com/Zamoca42/langchain-practice/assets/96982072/f551daff-fe48-4e35-9d73-2abb16d819b8)


# 3.3 

- OutputParser and LCEL

In [16]:
# Output Parser는 텍스트로만 응답하는 LLM의 응답형태를 변경
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()

p.parse("Hello,how, are, you?")

['Hello', 'how', 'are', 'you?']

In [25]:
template = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a list generating machine. Everything you ar asked will be answered with a comma seperated list of max {max_items} in korean. Do NOT reply with anything else."),
    ("human", "{question}"),
  ]
)

prompt = template.format_messages(max_items=10, question="What are the planets")

result = chat.predict_messages(prompt)

print(result)

p = CommaOutputParser()

p.parse(result.content)

content='수성, 금성, 지구, 화성, 목성, 토성, 천왕성, 해왕성'


['수성', '금성', '지구', '화성', '목성', '토성', '천왕성', '해왕성']

In [27]:
# chain 생성 prompt | model | output_parser

chain = template | chat | CommaOutputParser()

chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

['피카츄', '파이리', '꼬부기', '이상해씨', '리자드왕.']

# 3.4
- Chaining Chains

In [4]:
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.5)

chef_template = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
  ]
)

chef_chain = chef_template | chat

In [13]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."), 
    ("human", "{recipe}")
  ]
)

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
  "cuisine": "Italian", 
})

For a vegetarian version of Spaghetti Aglio e Olio, you can easily replace the Parmesan cheese with a vegetarian alternative such as nutritional yeast or a plant-based Parmesan-style cheese. These alternatives provide a similar savory and cheesy flavor to the dish without using animal products.

To prepare a plant-based Parmesan-style cheese, you can make a simple mixture using the following ingredients:

Ingredients:
- 1/2 cup raw cashews
- 2 tablespoons nutritional yeast
- 1/2 teaspoon garlic powder
- 1/2 teaspoon salt

Instructions:
1. In a food processor or blender, combine the raw cashews, nutritional yeast, garlic powder, and salt.
2. Pulse the mixture until it reaches a fine, crumbly texture similar to grated Parmesan cheese.
3. Taste and adjust the seasoning if needed.

You can sprinkle this plant-based Parmesan cheese alternative on top of your Spaghetti Aglio e Olio just like you would with traditional Parmesan cheese. It adds a delicious nutty and cheesy flavor to the dish w

AIMessageChunk(content='For a vegetarian version of Spaghetti Aglio e Olio, you can easily replace the Parmesan cheese with a vegetarian alternative such as nutritional yeast or a plant-based Parmesan-style cheese. These alternatives provide a similar savory and cheesy flavor to the dish without using animal products.\n\nTo prepare a plant-based Parmesan-style cheese, you can make a simple mixture using the following ingredients:\n\nIngredients:\n- 1/2 cup raw cashews\n- 2 tablespoons nutritional yeast\n- 1/2 teaspoon garlic powder\n- 1/2 teaspoon salt\n\nInstructions:\n1. In a food processor or blender, combine the raw cashews, nutritional yeast, garlic powder, and salt.\n2. Pulse the mixture until it reaches a fine, crumbly texture similar to grated Parmesan cheese.\n3. Taste and adjust the seasoning if needed.\n\nYou can sprinkle this plant-based Parmesan cheese alternative on top of your Spaghetti Aglio e Olio just like you would with traditional Parmesan cheese. It adds a deliciou

# 4.1

- FewShotPromptTemplate

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate

examples = [
  {
    "question": "What do you know about France?",
    "answer": """
    Here is what I know:
    Capital: Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro
    """,
  },
  {
    "question": "What do you know about Italy?",
    "answer": """
    I know this:
    Capital: Rome
    Language: Italian
    Food: Pizza and Pasta
    Currency: Euro
    """,
  },
  {
    "question": "What do you know about Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Food: Souvlaki and Feta Cheese
    Currency: Euro
    """,
  },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}\n")

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt, 
  examples=examples, 
  suffix="Human: What do you know about {country}?",
  input_variables=["country"]
)

# prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({
  "country": "Germany"
})

AI: 
    Here is what I know:
    Capital: Berlin
    Language: German
    Food: Bratwurst and Sauerkraut
    Currency: Euro

AIMessageChunk(content='AI: \n    Here is what I know:\n    Capital: Berlin\n    Language: German\n    Food: Bratwurst and Sauerkraut\n    Currency: Euro')

# 4.2

- FewShotChatMessagePromptTemplate

In [28]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

examples = [
  {
    "question": "France?",
    "answer": """
    Here is what I know:
    Capital: Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro
    """,
  },
  {
    "question": "Italy?",
    "answer": """
    I know this:
    Capital: Rome
    Language: Italian
    Food: Pizza and Pasta
    Currency: Euro
    """,
  },
  {
    "question": "Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Food: Souvlaki and Feta Cheese
    Currency: Euro
    """,
  },
]

example_prompt = ChatPromptTemplate.from_messages([
  ("human", "What do you know about {question}?"),
  ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt, 
  examples=examples, 
)

final_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a geography expert"),
  example_prompt,
  ("human", "What do you know about {country}?"),
])

chain = final_prompt | chat

chain.invoke({
  "country": "Thailand"
})



    I know this:
    Capital: Bangkok
    Language: Thai
    Food: Pad Thai and Tom Yum
    Currency: Thai Baht
    

AIMessageChunk(content='\n    I know this:\n    Capital: Bangkok\n    Language: Thai\n    Food: Pad Thai and Tom Yum\n    Currency: Thai Baht\n    ')

# 4.3

- LengthBasedExampleSelector

In [29]:
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.few_shot import FewShotPromptTemplate

examples = [
  {
    "question": "What do you know about France?",
    "answer": """
    Here is what I know:
    Capital: Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro
    """,
  },
  {
    "question": "What do you know about Italy?",
    "answer": """
    I know this:
    Capital: Rome
    Language: Italian
    Food: Pizza and Pasta
    Currency: Euro
    """,
  },
  {
    "question": "What do you know about Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Food: Souvlaki and Feta Cheese
    Currency: Euro
    """,
  },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}\n")

example_selector = LengthBasedExampleSelector(
  examples=examples,
  example_prompt=example_prompt,
  max_length=80,
)

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt, 
  example_selector=example_selector,
  suffix="Human: What do you know about {country}?",
  input_variables=["country"]
)

prompt.format(country="brazil")

'Human: What do you know about France?\nAI: \n    Here is what I know:\n    Capital: Paris\n    Language: French\n    Food: Wine and Cheese\n    Currency: Euro\n    \n\n\nHuman: What do you know about brazil?'

- RandomExampleSelector
  - 무작위 예제 선택

In [31]:
from langchain.prompts.example_selector.base import BaseExampleSelector

class RandomExampleSelector(BaseExampleSelector):
    
    def __init__(self, examples):
        self.examples = examples
        
    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice
        
        return [choice(self.examples)]


examples = [
  {
    "question": "What do you know about France?",
    "answer": """
    Here is what I know:
    Capital: Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro
    """,
  },
  {
    "question": "What do you know about Italy?",
    "answer": """
    I know this:
    Capital: Rome
    Language: Italian
    Food: Pizza and Pasta
    Currency: Euro
    """,
  },
  {
    "question": "What do you know about Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Food: Souvlaki and Feta Cheese
    Currency: Euro
    """,
  },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}\n")

example_selector = RandomExampleSelector(
  examples=examples,
)

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt, 
  example_selector=example_selector,
  suffix="Human: What do you know about {country}?",
  input_variables=["country"]
)

prompt.format(country="brazil")

'Human: What do you know about Greece?\nAI: \n    I know this:\n    Capital: Athens\n    Language: Greek\n    Food: Souvlaki and Feta Cheese\n    Currency: Euro\n    \n\n\nHuman: What do you know about brazil?'